In [47]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import *
import shutil
import difflib
from pulp import *
import openpyxl
import glob

#import datetime
today = date.today()
#today = today - datetime.timedelta(days=1)
today = today.strftime("%m.%d.20%y")
today

'03.16.2022'

In [48]:
## DKNG Player Pool

time = "0830_"
contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool'

shutil.move(downloads + "/"+ "DKSalaries.csv",
           contest_data_loc + "/"+ "DKSalaries.csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool")
dkng_file = shutil.copy("DKSalaries.csv", "DKSalaries_"+ contest + str(today)+".csv")
dkng_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries.csv")
dkng_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries_"+ contest + str(today)+".xlsx", index=False)
os.remove("DKSalaries.csv")
os.remove("DKSalaries_"+ contest + str(today)+".csv")

In [49]:
## Uploads
## Uploads
os.chdir(r"C:\Users\vchang\Downloads")

draftkings_upload = r"C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\DraftKings"

dk_upload = glob.glob('DKEntries.csv')[0]
os.rename(dk_upload, "DK_Upload_"+ contest + str(today)+".csv")

shutil.move(downloads + "/"+ "DK_Upload_"+ contest + str(today)+".csv",
           draftkings_upload + "/"+ "DK_Upload_"+ contest + str(today)+".csv")

'C:\\Users\\vchang\\Documents\\Fantasy Bball\\DFS\\Upload\\DraftKings/DK_Upload_PlayerPool_03.16.2022.csv'

In [50]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data/DraftKings'
daily_file = "DK NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+str(today)+".xlsx", index=False)
os.remove("DK NBA Projections.csv")

In [51]:
## Joins the DKNG and ETR Files to include DKNG Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\DraftKings\ETR_Daily_' + str(today) + ".xlsx")
dkng = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\DKNG_Data\PlayerPool\DKSalaries_PlayerPool_' + str(today) + ".xlsx")

## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              dkng[['Name','Roster Position', 'Name + ID']], 
              left_on='Player', 
              right_on='Name')

In [52]:
## Unnests Roster Positions into each Row (explode in Python) ##


## Update Salary to INT 
df['DK Salary'] = df['DK Salary'].str.replace(",","").astype(float)

df['Roster Position'] = df['Roster Position'].astype(str)
df['Roster Position'] = df['Roster Position'].str.split('/')

df = df.explode('Roster Position')

df.head()

df.to_csv("DKNG_final_PlayerPool_ETR_"+str(today)+".csv", index=False)
# os.system("DKNG_final_PlayerPool_ETR_"+str(today)+".xlsx")

In [53]:
df.head()

,Player,Team,Opp,Minutes,DK Position,DK Salary,DK Points,DK Value,DK Ceiling,DK Own,DK Slate,Name,Roster Position,Name + ID
0,Giannis Antetokounmpo,MIL,SAC,34.0,SF/PF,11700.0,59.3,6.5,68.0,28.6,MAIN,Giannis Antetokounmpo,SF,Giannis Antetokounmpo (21704689)
0,Giannis Antetokounmpo,MIL,SAC,34.0,SF/PF,11700.0,59.3,6.5,68.0,28.6,MAIN,Giannis Antetokounmpo,PF,Giannis Antetokounmpo (21704689)
0,Giannis Antetokounmpo,MIL,SAC,34.0,SF/PF,11700.0,59.3,6.5,68.0,28.6,MAIN,Giannis Antetokounmpo,F,Giannis Antetokounmpo (21704689)
0,Giannis Antetokounmpo,MIL,SAC,34.0,SF/PF,11700.0,59.3,6.5,68.0,28.6,MAIN,Giannis Antetokounmpo,UTIL,Giannis Antetokounmpo (21704689)
1,Domantas Sabonis,SAC,MIL,34.0,C,9200.0,47.2,4.4,55.6,4.0,MAIN,Domantas Sabonis,C,Domantas Sabonis (21704757)


In [54]:
os.getcwd()

'C:\\Users\\vchang\\Documents\\Fantasy Bball\\DFS\\PlayerPool Analysis'

In [55]:
df = pd.read_csv("DKNG_final_PlayerPool_ETR_"+str(today)+".csv", index_col=['Name + ID', 'Roster Position'], skipinitialspace=True)

legal_assignments = df.index # tuples of (name, pos)
name_set = df.index.unique(0) # a convenience

costs = df['DK Salary'].to_dict()
values = df['DK Points'].to_dict()

# set up LP
draft = pulp.LpVariable.dicts('selected', legal_assignments, cat='Binary')

prob = pulp.LpProblem('the draft', LpMaximize)

#obj
prob += pulp.lpSum([draft[n,p]*values[n,p] for (n,p) in legal_assignments])

#salary cap
prob += pulp.lpSum([draft[n,p]*costs[n,p] for (n, p) in legal_assignments]) <= 50000

# positions
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'PG']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'SG']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'G']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'SF']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'PF']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'F']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'C']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'UTIL']) == 1

# use each player at most only once
result = []
for name in name_set:
    prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if n == name]) <= 1

prob.solve()

for idx in draft:
    if draft[idx].varValue > 0:
        print(idx[0], idx[1])
        
        result.append([idx[0], idx[1]])

Giannis Antetokounmpo (21704689) PF
Jrue Holiday (21704590) G
Khris Middleton (21704615) SF
Fred VanVleet (21704685) UTIL
Justin Holiday (21704576) SG
Precious Achiuwa (21704948) C
Otto Porter Jr. (21704660) F
Davion Mitchell (21704888) PG


C:\Users\vchang\Anaconda3\lib\site-packages\pulp\pulp.py:1313: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [56]:
new_list = result
df = pd.DataFrame(new_list)
test = df[[1,0]]
test1 = test.transpose()
test2 = test1.rename(columns=test1.iloc[0])
test3 = test2.drop([1])
df2 = test3[['PG', 'SG','SF', 'PF', 'C', 'G', 'F', 'UTIL']]
df2.to_csv("dkng_result_" + str(today) + ".csv", index=None)
# os.system("dkng_result_" + str(today) + ".csv")

0

In [57]:
df2 = pd.read_csv(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis\dkng_result_' + str(today) + ".csv")

upload = pd.read_csv(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\DraftKings\DK_Upload_PlayerPool_' + str(today) + ".csv"
                      , error_bad_lines=False, engine ='python')

upload = upload[['Entry ID', 'Contest Name', 'Contest ID', 'Entry Fee']]

os.chdir(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\DraftKings')
result = pd.concat([upload, df2], axis=1)
result.to_csv("DK_Final_Upload_"+ time + str(today)+".csv", index=False)
os.system("DK_Final_Upload_"+ time + str(today)+".csv")

Skipping line 8: Expected 14 fields in line 8, saw 22
Skipping line 9: Expected 14 fields in line 9, saw 22
Skipping line 10: Expected 14 fields in line 10, saw 22
Skipping line 11: Expected 14 fields in line 11, saw 22
Skipping line 12: Expected 14 fields in line 12, saw 22
Skipping line 13: Expected 14 fields in line 13, saw 22
Skipping line 14: Expected 14 fields in line 14, saw 22
Skipping line 15: Expected 14 fields in line 15, saw 22
Skipping line 16: Expected 14 fields in line 16, saw 22
Skipping line 17: Expected 14 fields in line 17, saw 22
Skipping line 18: Expected 14 fields in line 18, saw 22
Skipping line 19: Expected 14 fields in line 19, saw 22
Skipping line 20: Expected 14 fields in line 20, saw 22
Skipping line 21: Expected 14 fields in line 21, saw 22
Skipping line 22: Expected 14 fields in line 22, saw 22
Skipping line 23: Expected 14 fields in line 23, saw 22
Skipping line 24: Expected 14 fields in line 24, saw 22
Skipping line 25: Expected 14 fields in line 25, saw

0

Skipping line 8: Expected 14 fields in line 8, saw 22
Skipping line 9: Expected 14 fields in line 9, saw 22
Skipping line 10: Expected 14 fields in line 10, saw 22
Skipping line 11: Expected 14 fields in line 11, saw 22
Skipping line 12: Expected 14 fields in line 12, saw 22
Skipping line 13: Expected 14 fields in line 13, saw 22
Skipping line 14: Expected 14 fields in line 14, saw 22
Skipping line 15: Expected 14 fields in line 15, saw 22
Skipping line 16: Expected 14 fields in line 16, saw 22
Skipping line 17: Expected 14 fields in line 17, saw 22
Skipping line 18: Expected 14 fields in line 18, saw 22
Skipping line 19: Expected 14 fields in line 19, saw 22
Skipping line 20: Expected 14 fields in line 20, saw 22
Skipping line 21: Expected 14 fields in line 21, saw 22
Skipping line 22: Expected 14 fields in line 22, saw 22
Skipping line 23: Expected 14 fields in line 23, saw 22
Skipping line 24: Expected 14 fields in line 24, saw 22
Skipping line 25: Expected 14 fields in line 25, saw

,Entry ID,Contest Name,Contest ID,Entry Fee,PG,SG,SF,PF,C,G,F,UTIL,Unnamed: 12,Instructions
0,3.160099e+09,NBA $2 50-50 (Turbo),124790434.0,$2,Jordan Clarkson (21704245),Tre Mann (21704513),Devin Vassell (21704458),Royce O'Neale (21704270),Jakob Poeltl (21704323),Donovan Mitchell (21704330),Rudy Gay (21704221),Shai Gilgeous-Alexander (21704448),NaN,1. Column A lists all of your contest entries ...
1,3.160089e+09,NBA $625 And-One [20 Entry Max] (Turbo),124818734.0,$1,Jordan Clarkson (21704245),Tre Mann (21704513),Devin Vassell (21704458),Royce O'Neale (21704270),Jakob Poeltl (21704323),Donovan Mitchell (21704330),Rudy Gay (21704221),Shai Gilgeous-Alexander (21704448),NaN,2. Your current lineup is listed next to each ...
2,3.160089e+09,NBA $1 Triple Up [Top 9 Win $3] (Turbo),124818876.0,$1,Jordan Clarkson (21704245),Tre Mann (21704513),Devin Vassell (21704458),Royce O'Neale (21704270),Jakob Poeltl (21704323),Donovan Mitchell (21704330),Rudy Gay (21704221),Shai Gilgeous-Alexander (21704448),NaN,3. You can change the lineup for an entry by e...
3,3.160089e+09,NBA $1 Double Up (Turbo),124829899.0,$1,Jordan Clarkson (21704245),Tre Mann (21704513),Devin Vassell (21704458),Royce O'Neale (21704270),Jakob Poeltl (21704323),Donovan Mitchell (21704330),Rudy Gay (21704221),Shai Gilgeous-Alexander (21704448),NaN,4. Use data from the Name+ID column or the ID ...
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5. For faster processing only include entries ...


,Entry ID,Contest Name,Contest ID,Entry Fee
0,3.160099e+09,NBA $2 50-50 (Turbo),124790434.0,$2
1,3.160089e+09,NBA $625 And-One [20 Entry Max] (Turbo),124818734.0,$1
2,3.160089e+09,NBA $1 Triple Up [Top 9 Win $3] (Turbo),124818876.0,$1
3,3.160089e+09,NBA $1 Double Up (Turbo),124829899.0,$1
4,NaN,NaN,NaN,NaN
